# Spectral Clustering  

In this notebook, we will use cuGraph to identify the cluster in a test graph using Spectral Clustering with both the (A) Balance Cut metric, and (B) the Modularity Maximization metric


Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Created:   08/01/2019
* Last Edit: 10/16/2019

RAPIDS Versions: 0.10.0

Test Hardware
* GV100 32G, CUDA 10.0



## Introduction

Spectral clustering uses the eigenvectors of a Laplacian of the input graph to find a given number of clusters which satisfy a given quality metric. Balanced Cut and Modularity Maximization are two such quality metrics. 

@See:  https://en.wikipedia.org/wiki/Spectral_clustering

To perform spectral clustering using the balanced cut metric in cugraph use:

__df = cugraph.spectralBalancedCutClustering(G, num_clusters, num_eigen_vects)__
<br>or<br>
__df = cugraph.spectralModularityMaximizationClustering(G, num_clusters, num_eigen_vects)__



Input
* __G__: A cugraph.Graph object
* __num_clusters__: The number of clusters to find
* __num_eig__: (optional) The number of eigenvectors to use

Returns
* __df__: cudf.DataFrame with two names columns:
    * df["vertex"]: The vertex id.
    * df["cluster"]: The assigned partition.

### cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon.

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)

Zachary used a min-cut flow model to partition the graph into two clusters, shown by the circles and squares.  Zarchary wanted just two cluster based on a conflict that caused the Karate club to break into two separate clubs.  Many social network clustering methods identify more that two social groups in the data.

In [1]:
# Import needed libraries
import cugraph
import cudf
import numpy as np

### Read the CSV datafile using cuDF

In [2]:
# Test file  
datafile='../data/karate-data.csv'

gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

### Adjusting the vertex ID
Let's adjust all the vertex IDs to be zero based.  We are going to do this by creating two new columns with the adjusted IDs

In [3]:
gdf["src"] = gdf["src"] - 1
gdf["dst"] = gdf["dst"] - 1

In [4]:
# The algorithm requires that there are edge weights.  In this case all the weights are being ste to 1
gdf["data"] = cudf.Series(np.ones(len(gdf), dtype=np.float32))

In [5]:
# Look at the first few data records - the output should be two colums src and dst
gdf.head()

,src,dst,data
0,0,1,1.0
1,0,2,1.0
2,0,3,1.0
3,0,4,1.0
4,0,5,1.0


In [6]:
# verify data type
gdf.dtypes

src       int32
dst       int32
data    float32
dtype: object

Everything looks good, we can now create a graph

In [7]:
# create a CuGraph 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst', edge_attr='data')

----
#### Define and print function, but adjust vertex ID so that they match the illustration

In [8]:
def print_cluster(_df, id):
    
    _f = _df.query('cluster == @id')
  
    part = []
    for i in range(len(_f)):
        part.append(_f['vertex'][i] + 1)
    print(part)

----
#### Using Balanced Cut

In [9]:
# Call spectralBalancedCutClustering on the graph for 3 clusters
# using 3 eigenvectors:
bc_gdf = cugraph.spectralBalancedCutClustering(G, 3, num_eigen_vects=3)

In [10]:
# Check the edge cut score for the produced clustering
score = cugraph.analyzeClustering_edge_cut(G, 3, bc_gdf['cluster'])
score

31.0

In [11]:
# See which nodes are in cluster 0:
print_cluster(bc_gdf, 0)

[4, 10, 13, 15, 16, 19, 20, 21, 22, 23, 27, 33]


In [12]:
# See which nodes are in cluster 1:
print_cluster(bc_gdf, 1)

[25, 26]


In [13]:
# See which nodes are in cluster 2:
print_cluster(bc_gdf, 2)

[1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 14, 17, 18, 24, 28, 29, 30, 31, 32, 34]


----
#### Modularity Maximization
Let's now look at the clustering using the modularity maximization metric

In [14]:
# Call spectralModularityMaximizationClustering on the graph for 3 clusters
# using 3 eigenvectors:
mm_gdf = cugraph.spectralModularityMaximizationClustering(G, 3, num_eigen_vects=3)

In [15]:
# Check the modularity score for the produced clustering
score = cugraph.analyzeClustering_modularity(G, 3, mm_gdf['cluster'])
score

0.3579881489276886

In [16]:
# See which nodes are in cluster 0:
print_cluster(mm_gdf, 0)

[9, 10, 15, 16, 19, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]


In [17]:
print_cluster(mm_gdf, 1)

[17]


In [18]:
print_cluster(mm_gdf, 2)

[1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 18, 20, 22]


Notice that the two metrics produce different results

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___